## 🔹 Cell 0: Imports and global guards

#### **Cell 0: Environment Verification**

In [ ]:
import time
import numpy as np
import cupy as cp
from cuml.linear_model import LogisticRegression

# Verify A100 is visible
print(f"GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
cp.cuda.runtime.deviceSynchronize()

#### **Cell 1: Synthetic Data Setup (A100 Scale)**
We will use 40,000 rows and 25,132 features to simulate a heavy portion of the test set.

In [ ]:
n_samples = 40000
n_features = 25132
n_targets = 500  # Size of one TARGET_CHUNK

# Materialize features directly on GPU
X_gpu = cp.random.standard_normal((n_samples, n_features), dtype=cp.float32)
W_gpu = cp.random.standard_normal((n_targets, n_features), dtype=cp.float32)
b_gpu = cp.random.standard_normal((n_targets,), dtype=cp.float32)

print(f"X_gpu Memory: {X_gpu.nbytes / 1e9:.2f} GB")
cp.cuda.runtime.deviceSynchronize()

#### **Cell 2: Test 1 - The "Slow" Path (Batch Synchronization)**
This simulates your current code calling `.get()` or `_to_numpy` inside the loop.

In [ ]:
batch_size = 4096
host_buffer = np.zeros((n_samples, n_targets), dtype=np.float32)

t0 = time.time()
for i in range(0, n_samples, batch_size):
    i1 = min(i + batch_size, n_samples)
    xb = X_gpu[i:i1, :]
    
    # Simulate LogReg predict_proba
    # This .get() call forces the CPU to stop and wait for the GPU
    p_batch = (cp.dot(xb, W_gpu.T) + b_gpu).get() 
    
    host_buffer[i:i1, :] = p_batch

cp.cuda.runtime.deviceSynchronize()
print(f"❌ Slow Path (with sync): {time.time() - t0:.4f} seconds")

#### **Cell 3: Test 2 - The "Fast" Path (Asynchronous Pipelining)**
This represents the Rank 1 fix: keeping everything on GPU and doing one transfer at the end.

In [ ]:
gpu_buffer = cp.zeros((n_samples, n_targets), dtype=cp.float32)

t0 = time.time()
for i in range(0, n_samples, batch_size):
    i1 = min(i + batch_size, n_samples)
    xb = X_gpu[i:i1, :]
    
    # Asynchronous GPU operation (No .get() here)
    gpu_buffer[i:i1, :] = cp.dot(xb, W_gpu.T) + b_gpu

# One single synchronization and transfer after the whole chunk is done
final_host_output = gpu_buffer.get() 

cp.cuda.runtime.deviceSynchronize()
print(f"✅ Fast Path (pipelined): {time.time() - t0:.4f} seconds")

### **What this Notebook Proves**
1.  **Isolation of the `.get()` overhead:** Cell 2 will be significantly slower than Cell 3. On an A100, the constant handshaking between the CPU and GPU during batch writes is the primary "kink in the hose".
2.  **PCIe Saturation:** Cell 3 allows the GPU to queue all matrix multiplications. The only delay is the final transfer of the 500-column chunk to RAM, which is the most efficient way to use the PCIe bus.
3.  **Memmap Impact:** By performing the transfer in one large block, you can then write to the `test_pred` memmap in one sequential sweep, avoiding the random-access disk latency that is likely contributing to your 80-minute runtime [Log, Conversation History].

**Analogy for the Test:**
Cell 2 is like a worker **unloading a truck one box at a time**, walking it into the warehouse, and coming back for the next. Cell 3 is like using a **conveyor belt**: you let the boxes flow continuously and only check the total inventory once the truck is empty. For 224,309 proteins, the conveyor belt is the only way to hit your 10-hour target.

In [ ]:
import time
import inspect
import os
from pathlib import Path
import numpy as np
import cupy as cp
import cuml
from cuml.multiclass import OneVsRestClassifier
from cuml.linear_model import LogisticRegression

from cupy.cuda import runtime

cp.cuda.runtime.setDevice(0)
cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)

print("CuPy:", cp.__version__)
print("cuML:", cuml.__version__)
print("CUDA Runtime:", runtime.runtimeGetVersion())
print("GPU:", runtime.getDeviceProperties(0)["name"])

## 🔹 Cell 0.5: Setup Data and Model (Auto-fill)

This cell ensures `X_test_gpu` and `clf_chunk` exist so the diagnostics can run standalone.

In [ ]:
# Try to find data
candidates = [
    Path.cwd() / 'cafa6_data',
    Path('../cafa6_data'),
    Path('../../cafa6_data'),
    Path('c:/Users/Olale/Documents/Codebase/Science/cafa-6-protein-function-prediction/cafa6_data')
]
WORK_ROOT = next((p for p in candidates if (p / 'features' / 'X_test_mmap.npy').exists()), None)

if WORK_ROOT:
    print(f"Found data at: {WORK_ROOT}")
    xt_path = WORK_ROOT / 'features' / 'X_test_mmap.npy'
    X_test_host = np.load(xt_path, mmap_mode='r')
    
    # Load a chunk to GPU (enough for diagnostics)
    # We load 40k rows to ensure we have enough data for the batch size sweeps
    n_rows = min(X_test_host.shape[0], 40000)
    print(f"Loading {n_rows} rows of X_test to GPU...")
    X_test_gpu = cp.asarray(X_test_host[:n_rows], dtype=cp.float32)
    
    # Fit a dummy model to simulate the trained clf_chunk
    print("Fitting dummy cuML model for inference testing...")
    n_classes = 50
    y_dummy = cp.random.randint(0, 2, (n_rows, n_classes)).astype(cp.float32)
    clf_chunk = OneVsRestClassifier(LogisticRegression(output_type='cupy'))
    clf_chunk.fit(X_test_gpu, y_dummy)
    
else:
    print("⚠️ Data not found. Generating SYNTHETIC data for diagnostics.")
    X_test_gpu = cp.random.randn(10000, 1484, dtype=cp.float32)
    y_dummy = cp.random.randint(0, 2, (10000, 50)).astype(cp.float32)
    clf_chunk = OneVsRestClassifier(LogisticRegression(output_type='cupy'))
    clf_chunk.fit(X_test_gpu, y_dummy)

print("✅ Setup complete. `X_test_gpu` and `clf_chunk` are ready.")

## 🔹 Cell 1: Hard assertion that GPU is live

In [ ]:
x = cp.zeros((1024, 1024), dtype=cp.float32)
t0 = time.time()
cp.cuda.runtime.deviceSynchronize()
print("GPU sync latency:", time.time() - t0)

assert isinstance(x, cp.ndarray)

This should be essentially instantaneous.

## 🔹 Cell 2: Inspect `X_test_gpu` residency and layout

In [ ]:
X = X_test_gpu  # must already exist

print("Type:", type(X))
print("Device:", X.device)
print("Shape:", X.shape)
print("Dtype:", X.dtype)
print("C contiguous:", X.flags.c_contiguous)
print("Strides:", X.strides)

# Slice timing
t0 = time.time()
xb = X[0:8192, :]
cp.cuda.runtime.deviceSynchronize()
print("Slice time (8k rows):", time.time() - t0)

**Expected**

* `cp.ndarray`
* `float32`
* `C contiguous = True`
* Slice time: milliseconds

## 🔹 Cell 3: Confirm slicing is a view, not a copy

In [ ]:
xb = X[0:1024, :]

print("xb.base is None:", xb.base is None)
print("Pointer equality:", xb.data.ptr == X.data.ptr)

If `base is None` or pointers differ, every batch is copying on GPU.

That alone explains your runtime.

## 🔹 Cell 4: Inspect the trained model object

In [ ]:
clf = clf_chunk  # must already exist

print("Model type:", type(clf))

if hasattr(clf, "coef_"):
    print("coef_ type:", type(clf.coef_))
    print("coef_ dtype:", clf.coef_.dtype)
    print("coef_ device:", getattr(clf.coef_, "device", "CPU"))

if hasattr(clf, "intercept_"):
    print("intercept_ type:", type(clf.intercept_))

**Hard requirements**

* `coef_` must be CuPy
* `float32`
* On GPU

Anything else risks CPU fallback.

## 🔹 Cell 5: Is `predict_proba` actually fast on GPU?

This isolates cuML itself.

In [ ]:
import time
import cupy as cp

# -----------------------------
# Select batch (must be a view)
# -----------------------------
xb = X_test_gpu[0:8192, :]

# Sanity checks (fail fast)
assert isinstance(xb, cp.ndarray), "xb is not on GPU"
assert xb.dtype == cp.float32, f"Unexpected dtype: {xb.dtype}"
assert xb.flags.c_contiguous, "xb is not C-contiguous"

def safe_predict_proba(clf, x):
    """
    Robust prediction that tries manual GEMM first to bypass cuML Python overhead/bugs.
    This is ideal for diagnostics as it measures raw GPU throughput.
    """
    W = None
    b = None

    # Attempt to get W and b
    if hasattr(clf, 'coef_') and hasattr(clf, 'intercept_'):
        W = clf.coef_
        b = clf.intercept_
    elif hasattr(clf, 'estimators_'):
        # Fallback: reconstruct from OVR estimators
        try:
            # Collect coefs, handling potential device mismatch
            ws = []
            bs = []
            for i, e in enumerate(clf.estimators_):
                if not hasattr(e, 'coef_'):
                    # Some versions of cuML OVR might store estimators differently
                    continue
                
                w = e.coef_
                b_val = e.intercept_
                
                # Convert to CuPy if needed
                if not isinstance(w, cp.ndarray): w = cp.asarray(w)
                if not isinstance(b_val, cp.ndarray): b_val = cp.asarray(b_val)
                
                ws.append(w)
                bs.append(b_val)
            
            if ws:
                W = cp.vstack(ws)
                b = cp.hstack(bs)
            else:
                print("No estimators with coef_ found.")
                
        except Exception as e:
            print(f"Could not reconstruct coefs from estimators: {e}")

    # Strategy 1: Manual GEMM
    if W is not None and b is not None:
        try:
            # Ensure CuPy
            if not isinstance(W, cp.ndarray): W = cp.asarray(W)
            if not isinstance(b, cp.ndarray): b = cp.asarray(b)
            
            if W.ndim == 1:
                scores = cp.dot(x, W) + b
            else:
                scores = cp.dot(x, W.T) + b
                
            return 1.0 / (1.0 + cp.exp(-cp.clip(scores, -50.0, 50.0)))
        except Exception as e:
            print(f"Manual GEMM failed: {e}")

    # Strategy 2: decision_function
    if hasattr(clf, 'decision_function'):
        try:
            scores = clf.decision_function(x)
            if not isinstance(scores, cp.ndarray):
                 scores = cp.asarray(scores)
            return 1.0 / (1.0 + cp.exp(-cp.clip(scores, -50.0, 50.0)))
        except Exception:
            pass

    # Strategy 3: predict_proba
    if hasattr(clf, 'predict_proba'):
        try:
            return clf.predict_proba(x)
        except Exception:
            pass
        
    # Strategy 4: predict (Last resort, returns hard classes but keeps pipeline moving)
    print("⚠️ Warning: Falling back to hard `predict()` (probabilities unavailable).")
    try:
        return clf.predict(x)
    except TypeError:
        # Fallback for sklearn-based predict that can't handle GPU arrays
        return cp.asarray(clf.predict(x.get()))

# -----------------------------
# Warm-up (kernel + cache)
# -----------------------------
_ = safe_predict_proba(clf_chunk, xb)
cp.cuda.runtime.deviceSynchronize()

# -----------------------------
# Timed run
# -----------------------------
t0 = time.time()
p = safe_predict_proba(clf_chunk, xb)
cp.cuda.runtime.deviceSynchronize()
dt = time.time() - t0

# -----------------------------
# Report
# -----------------------------
print(f"predict_proba (via manual GEMM/fallback) time (8192 rows): {dt:.6f} s")
print("Output type:", type(p))
print("Output device:", getattr(p, "device", "CPU"))
print("Output shape:", p.shape)
print("Output dtype:", p.dtype)

If this is **> 0.1s**, something is wrong already.

## 🔹 Cell 6: Batch size scaling sanity check

In [ ]:
for bs in [512, 1024, 4096, 8192, 16384]:
    if bs > X.shape[0]:
        print(f"BS={bs:<6} skipped (not enough data)")
        continue
    xb = X[0:bs, :]
    t0 = time.time()
    _ = safe_predict_proba(clf, xb)
    cp.cuda.runtime.deviceSynchronize()
    print(f"BS={bs:<6} time={time.time()-t0:.4f}s")

You want roughly linear scaling.
Flat or erratic scaling = launch bound or CPU fallback.

## 🔹 Cell 7: Inspect `_predict_proba_like` for hidden CPU paths

In [ ]:
# Define the function if it's not in scope (it was a helper in the other notebook)
# We'll just inspect the one from the user's notebook if we can, or define a dummy one to check logic
# Since we can't easily inspect a function from another notebook without running it, 
# I will paste the function here so we can inspect IT, assuming this is the one being used.

def _to_numpy(a):
    if hasattr(a, 'get'):
        return a.get()
    if hasattr(a, 'to_numpy'):
        return a.to_numpy()
    return np.asarray(a)

def _sigmoid_np(z):
    z = np.asarray(z, dtype=np.float32)
    z = np.clip(z, -50.0, 50.0)
    return 1.0 / (1.0 + np.exp(-z))

def _ensure_2d(p):
    p = np.asarray(p)
    if p.ndim == 1:
        p = p.reshape(-1, 1)
    return p

def _predict_proba_like(clf, x, expect_cols: int | None = None):
    # RAPIDS/cuML OVR may not implement predict_proba; fall back to decision_function->sigmoid.
    if hasattr(clf, 'predict_proba'):
        p = _ensure_2d(_to_numpy(clf.predict_proba(x)))
        # Binary single-label can come back as (n,2); we need P(class=1)
        if expect_cols == 1 and p.shape[1] == 2:
            p = p[:, 1:2]
        return np.asarray(p, dtype=np.float32)
    if hasattr(clf, 'decision_function'):
        s = _ensure_2d(_to_numpy(clf.decision_function(x)))
        return np.asarray(_sigmoid_np(s), dtype=np.float32)
    # Last resort: hard predictions (0/1)
    p = _ensure_2d(_to_numpy(clf.predict(x)))
    return np.asarray(p, dtype=np.float32)

print(inspect.getsource(_predict_proba_like))

You are looking for **any** of:

* `np.asarray`
* `cp.asnumpy`
* Python loops over columns
* Shape fixing logic

If NumPy appears here, that’s your kink.

## 🔹 Cell 8: Detect hidden synchronisation via memory deltas

In [ ]:
free_before = cp.cuda.runtime.memGetInfo()[0]

# Use safe_predict_proba to avoid cuML bugs
_ = safe_predict_proba(clf, X[0:8192, :])
cp.cuda.runtime.deviceSynchronize()

free_after = cp.cuda.runtime.memGetInfo()[0]

print("Free memory delta:", free_before - free_after)

Large or inconsistent deltas indicate implicit copies or reallocations.

## 🔹 Cell 9: Diagnose the write path explicitly

This is a prime suspect.

In [ ]:
# Using safe_predict_proba to bypass cuML AttributeError
p_gpu = safe_predict_proba(clf, X[0:2048, :])
cp.cuda.runtime.deviceSynchronize()

chunk_preds_ram = np.zeros((2048, p_gpu.shape[1]), dtype=np.float32)

t0 = time.time()
# Explicitly transfer to host to avoid implicit conversion error
chunk_preds_ram[:, :] = p_gpu.get()
print("GPU → RAM write time:", time.time() - t0)

If this takes more than a few milliseconds, you are synchronising per batch in the real loop.

## 🔹 Cell 10: End-to-end mini loop simulation (NO TRAINING)

This simulates your test loop structure.

In [ ]:
print("Running fixed Cell 10 with safe_predict_proba...")
TEST_BS = 4096
n = 16384

chunk_preds_ram = np.zeros((n, p_gpu.shape[1]), dtype=np.float32)

t0 = time.time()
for b0 in range(0, n, TEST_BS):
    b1 = min(b0 + TEST_BS, n)
    xb = X[b0:b1, :]
    
    # Use safe_predict_proba to avoid cuML bugs
    p = safe_predict_proba(clf, xb)
    
    # Explicitly transfer to host to avoid implicit conversion error
    chunk_preds_ram[b0:b1, :] = p.get()

cp.cuda.runtime.deviceSynchronize()
print("Mini loop total time:", time.time() - t0)

If this already feels slow, scaling to 224k rows explains your 80 minutes.

## 🔹 Cell 11: Final verdict print

In [ ]:
print("""
If inference is slow:
- Check Cell 3 (view vs copy)
- Check Cell 5 (raw predict speed)
- Check Cell 7 (_predict_proba_like CPU usage)
- Check Cell 9 (write synchronisation)

Do NOT proceed to full training until all four are clean.
""")

## What I expect we’ll find

I’ll be blunt and precise:

* You are almost certainly **syncing on every batch write**.
* Or `_predict_proba_like` is **doing CPU column alignment**.
* Or `X_test_gpu` is **not truly contiguous** anymore.

Any one of these explains the regression. Two together guarantee it.

## Next move

Run this notebook **once**, no training, no long jobs.
Paste back:

* Cell 3 output
* Cell 5 timing
* Cell 7 source
* Cell 9 timing

From there, I’ll tell you exactly which lines to delete or rewrite so the A100 finally behaves like an A100.